A running example
================

Algorithm on series (*e.g.* time series, character strings, ...)

A serie can be either a one dimensional ordered serie of items (e.g. an numerical array, a string).

We want to compute a dissimilarity measure between series. The measure can either apply to series of 
same length or not, and can be a metric (i.e. symetric, d(x, y) = 0 <=> x = y, triangular inequality). 

We consider two dissimilarity measures with different features. 

S1 and S2 two series of length |S1| and |S2|

- *dtw* dynamic time wrapping, complexity O(|S1|*|S2|)
- *cort* normalized cosine distances between derivatives, complexity O(|S1| + |S2|)



Dynamic Time Wrapping 
------------------------------------

- **Input:** two series, S1 and S2, not necessarilly of same length
- **Output:** a dissimilarity measure
- **Complexity:** O([S1|*|S2|)
- **Metric:** no: does not respect the triangular inequality
- **Side product:** an alignment between the series can be stored

What do we compute: 
-------------------------------
The transformation (with minimal cost) to transform one serie in the other one. 


Example of what is computed
-------------------------------------------



Example of result of dtw :
-----------------------------------

<div align="middle">
<img src="./fig/dtw_example.png"  style="width: 100%">
</div>


Idea of the algorithm
------------------------------

Inspired from 
https://riptutorial.com/dynamic-programming/example/25780/introduction-to-dynamic-time-warping


Let a and b two series. We have: 

- dtw is a dynamic programming algorithm: the solution is built incrementally
- a table t is incrementally filled.
- the value of the cell t[i, j] holds the *distance* between the sub series a[:i] and b[:j] 
- the value of the cell t[i, j] is computed using the values of cells t[i-1, j], t[i, j-1] and t[i-1, j-1]: 

t[i, j] = d(i, j) + min(t[i-1, j], t[i-1, j-1], t[i, j-1])

where d(i, j) is the distance between s[i] and s[j] (we will use the absolute difference)


An example with two series [0, 1, 1, 2, 2, 3, 5] and [0, 1, 2, 3, 5, 5, 5, 6] 

(image from https://riptutorial.com/dynamic-programming/example/25780/introduction-to-dynamic-time-warping

<div align="middle">
<img src="./fig/dtw_ex_table.jpg"  style="width: 100%">
</div>

why 6 in the t[-1, -1] ? 


Easy enought to implement:
----------------------------------------

In [14]:
import numpy as np
def DTWDistance_pure_python(s1, s2):
    """ Computes the dtw between s1 and s2 with distance the absolute distance

    :param s1: the first serie (ie an iterable over floats64)
    :param s2: the second serie (ie an iterable over floats64)
    :returns: the dtw distance
    :rtype: float64
    """
    _dtw_mat = np.empty([len(s1), len(s2)])
    _dtw_mat[0, 0] = abs(s1[0] - s2[0])

    #  two special cases : filling first row and columns
    for j in range(1, len(s2)):
        dist = abs(s1[0]-s2[j])
        _dtw_mat[0, j] = dist + _dtw_mat[0, j-1]

    for i in range(1, len(s1)):
        dist = abs(s1[i]-s2[0])
        _dtw_mat[i, 0] = dist + _dtw_mat[(i-1, 0)]

    # filling the matrix
    for i in range(1, len(s1)):
        for j in range(1, len(s2)):
            dist = abs(s1[i]-s2[j])
            _dtw_mat[(i, j)] = dist + min(_dtw_mat[i-1, j],
                                          _dtw_mat[i, j-1],
                                          _dtw_mat[i-1, j-1])

    return _dtw_mat[len(s1)-1, len(s2)-1], _dtw_mat


In [21]:

x = [1, 2, 3, 5, 5, 5, 6]
y = [1, 1, 2, 2, 3, 5]
nx = len(x)
ny = len(y)

d, mat = DTWDistance_pure_python(x, y)
d, mat

(1.0, array([[ 0.,  0.,  1.,  2.,  4.,  8.],
        [ 1.,  1.,  0.,  0.,  1.,  4.],
        [ 3.,  3.,  1.,  1.,  0.,  2.],
        [ 7.,  7.,  4.,  4.,  2.,  0.],
        [11., 11.,  7.,  7.,  4.,  0.],
        [15., 15., 10., 10.,  6.,  0.],
        [20., 20., 14., 14.,  9.,  1.]]))


Cort 
-------

**Input**: two series S1 and S2 *of same length*

**Output:** a dissimilarity measure

**Complexity:** O(|S1|+|S2|)

**Metric:** yes

What do we compute: 
-------------------------------

cosine distance between derivatives. 

normalize 

    num = 0.0
    sum_square_x = 0.0
    sum_square_y = 0.0
    for t in range(len(s1) - 1):
        slope_1 = s1[t + 1] - s1[t]
        slope_2 = s2[t + 1] - s2[t]
        num += slope_1 * slope_2
        sum_square_x += slope_1 * slope_1
        sum_square_y += slope_2 * slope_2
    return num / (np.sqrt(sum_square_x * sum_square_y))




What do we compute:
------------------------------

<div align="middle">
<img src="./fig/cort_formule.png"  style="width: 75%">
</div>

Easy enought to implement:
---------------------------------------

In [28]:
from math import sqrt

def cort(s1,  s2):
    """ Computes the cort between serie one and two (assuming they have the same length)

    :param s1: the first serie (or any iterable over floats64)
    :param s2: the second serie (or any iterable over floats64)
    :returns: the cort distance
    :rtype: float
    :precondition: series are assumed to be of same size

    """
    num = 0.0
    sum_square_x = 0.0
    sum_square_y = 0.0
    for t in range(len(s1)-1):
        slope_1 = s1[t+1] - s1[t]
        slope_2 = s2[t+1] - s2[t]
        num = num + slope_1 * slope_2
        sum_square_x = sum_square_x + (slope_1*slope_1)
        sum_square_y = sum_square_y + (slope_2 * slope_2)
    return num/(sqrt(sum_square_x*sum_square_y))

In [29]:
x = [1, 2, 3, 5, 5, 6]
y = [1, 1, 2, 2, 3, 5]

cort(x,y)

0.4629100498862757